In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [4]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.2 MB/s 


In [8]:
!head -n 20 Poil_gapClosed.fa

>scaffold1_cov228
TTTAGCACCTCCTATTACGCGACCGGAATTAGTTTGCCACTACACTGAAACACAAAAGTGTGCTGTACTGTATGGACCTG
CAGGAGCCGGAAAATCTCATATCCTAGGGCAGATTGCAAAGCAAAGAGTCAGTGAGGGATTACCGACTATTTTAGTATTG
GGACAAAGCCTTTCAAACACGCTTTTTTGGGAACAAATCAGTGGTGTACTAGGCTTGGAAGGCAGAACAATGGATGATAT
TCTAGGAAATTTGAATATCGCAGGTGAGCGTAGAGGAGTACGCACTGTTATCCTATTTGACGCGATCAACGAAGGAGTTG
GAGCGCATTACTGGAGACAAAATTTATCTGAAATCGTTAGCGCAATTCAAAAATATTCACATTTGGCAGCCGTATTCTCA
TGTCGCGAAGAATACCTTTCATATGCATTTCCTGAATCCTTATCTGAAAAACTACCTAAATTATTCATCAATGGCTTTTC
AAATCCCGATGAGCTTGAGCGAGCAGCAATCCAATACCTTGATGAGAAGGGAATAGCTCGCCCCAATACACCGTGGCTTT
CTCCTGAATTTAGCAACCCTCTCTTTCTAAAAACGGCAAGTGAGGCTTTACTAGCTAAAGGGCTTACTGAATCGCCCCTA
GTTTCCTAGACACCCATTTGCTTCAAGAAA
>scaffold2_cov236
TTTAGCACCTCCTATTACGCGACCGGAATTAGTTTGCCACTACAGAACATTCAACATCGACAGCATTATCATCATTTATT
AATTTAAACTTTTCCTCCATATCTCGCGCACGTCGAGAAAGACGAACGGCTCCCATTGTTCCGGCAGCACCCTTTAACGT
ATGTAGGGCTTTCGTGCTTTCATGCCATTGTTTAGTCTGCCAACACAGAGCAACTTCCTCAATGATTGTTTGCATTTCTA
CTACAAACATACTCTGCATTCTTTTAAATAGGTTCTTATTAT

In [19]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [20]:
scaf=dict()
for record in SeqIO.parse("/content/Poil_gapClosed.fa", "fasta"):
  record.annotations['molecule_type'] = 'DNA'
  record.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()
  record.annotations['data_file_division'] = 'BCT'
  scaf[record.id] = record

In [26]:
genes = dict()
for gene in SeqIO.parse("proteins.fasta", "fasta"):
  descri = gene.description.split(' ')
  if descri[4] == '+':strand = 1 
  else: strand = -1
  featur = SeqFeature(FeatureLocation(int(descri[2]), int(descri[3]), strand=strand), type="CDS")
  featur.qualifiers['locus_tag'] = [descri[0]]
  featur.qualifiers['translation'] = [gene.seq]
  nm=descri[1].split('_')
  scaf_name=nm[0]+'_'+nm[2]
  scaf[scaf_name].features.append(featur)
  genes[descri[0]] = featur

In [27]:
featur.qualifiers

OrderedDict([('locus_tag', ['3571']),
             ('translation',
              [Seq('GLPLYRMETIFKRMGIHLPRNTLANWMIKSSECLQPLYNLLNDQLLESGYLHMD...DDY')])])

In [37]:
mil_1_g = dict()
for mil_1_f in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if 'protein_id' not in mil_1_f.qualifiers or 'product' not in mil_1_f.qualifiers:
    continue

  mil_1_g[mil_1_f.qualifiers['protein_id'][0]] = mil_1_f.qualifiers['product'][0]

In [38]:
names = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
mil_1_h = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=names)
mil_1_h

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,2,lcl|HF680312.1_prot_CCU72326.1_1877,99.054,317,3,0,3,319,1,317,0.000000e+00,644.0
1,2,lcl|HF680312.1_prot_CCU72392.1_1943,34.583,240,138,5,85,307,956,1193,2.330000e-30,118.0
2,2,lcl|HF680312.1_prot_CCU72759.1_2310,27.667,300,188,8,8,302,709,984,1.020000e-24,101.0
3,2,lcl|HF680312.1_prot_CCU70543.1_94,42.857,126,71,1,79,204,798,922,5.630000e-24,99.4
4,2,lcl|HF680312.1_prot_CCU70690.1_241,26.140,329,205,10,2,303,591,908,6.260000e-23,96.7
...,...,...,...,...,...,...,...,...,...,...,...,...
10075,3571,lcl|HF680312.1_prot_CCU73517.1_3068,96.552,116,4,0,1,116,216,331,4.090000e-80,241.0
10076,3571,lcl|HF680312.1_prot_CCU73713.1_3264,96.552,116,4,0,1,116,216,331,5.770000e-80,241.0
10077,3571,lcl|HF680312.1_prot_CCU73876.1_3427,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0
10078,3571,lcl|HF680312.1_prot_CCU73197.1_2748,96.552,116,4,0,1,116,223,338,8.800000e-80,241.0


In [39]:
hits = mil_1_h[mil_1_h['sseqid'].str.contains("CCU")].sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
hits

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
5648,2065,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
6997,2521,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
2034,734,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
5475,1994,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
2831,1012,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2770,981,lcl|HF680312.1_prot_CCU71655.1_1206,93.750,32,2,0,21,52,220,251,1.050000e-12,56.6
2114,778,lcl|HF680312.1_prot_CCU73367.1_2918,41.096,73,43,0,5,77,8,80,7.960000e-13,54.7
2821,1004,lcl|HF680312.1_prot_CCU71629.1_1180,61.224,49,8,1,2,39,139,187,1.280000e-11,54.7
10022,3562,lcl|HF680312.1_prot_CCU72581.1_2132,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [40]:
for i, hit in hits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('_')[2]
  gene.qualifiers['product'] = [mil_1_g[match]]

In [32]:
!wget -nc https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
!gzip -d uniprot_sprot.dat.gz

--2022-10-18 20:27:52--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 642093634 (612M) [application/x-gzip]
Saving to: ‘uniprot_sprot.dat.gz’

uniprot_sprot.dat.g 100%[===================>] 612.35M   121MB/s    in 4.9s    

2022-10-18 20:27:57 (124 MB/s) - ‘uniprot_sprot.dat.gz’ saved [642093634/642093634]



In [33]:
!grep '^ID\|^DE   RecName: Full=' uniprot_sprot.dat > SwissProt_names.txt

In [34]:
previd = None
swgenes = dict()
for line in open('SwissProt_names.txt'):
  if line.startswith('ID'):previd = line.split()[1]
  if line.startswith('DE'):swgenes[previd] = line.split('=')[1][:-2]

In [35]:
swhits_raw = pd.read_csv('scaffolds.hits_from_SwissProt.txt', sep='\t', header=None, names=names)
swhits = swhits_raw.sort_values('bitscore', ascending=False).drop_duplicates('qseqid')
for i, hit in swhits.iterrows():
  gene = genes[str(hit['qseqid'])]
  match = hit['sseqid'].split('|')[-1]
  gene.qualifiers['product'] = [swgenes[match]]

In [36]:
SeqIO.write(scaf.values(), "GENOME.gbk", "genbank")

71